In [7]:
!pip install -q transformers

In [8]:
from transformers import pipeline

In [1]:
import pandas as pd
import string
from bs4 import BeautifulSoup
from urllib.request import urlopen
from itertools import combinations
import re
import os
from pathlib import Path


from bs4 import BeautifulSoup
from urllib.request import urlopen



def get_characters(url):
    characters = []
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    images = soup.find_all("img")

    for image in images:
        name = image["alt"]
        name = name.split(" ")[0]
        if name not in characters:
            characters.append(name)
    return characters


book_characters = {
    "Blood of Elves.txt" : "https://witcher.fandom.com/wiki/Category:Blood_of_Elves_characters",
    "Times of Contempt.txt" : "https://witcher.fandom.com/wiki/Category:Time_of_Contempt_characters",
    "Baptism of Fire.txt" : "https://witcher.fandom.com/wiki/Category:Baptism_of_Fire_characters",
    "The Tower of the Swallow.txt" : "https://witcher.fandom.com/wiki/Category:The_Tower_of_the_Swallow_characters",
    "The Lady of the Lake.txt" : "https://witcher.fandom.com/wiki/Category:The_Lady_of_the_Lake_characters",
}


In [2]:
def get_pairwise_characters(link):

    characters = get_characters(link)

    pairwise_characters = list(combinations(characters, 2))
    pairwise_characters_list = [list(pair) for pair in pairwise_characters]

    return pairwise_characters_list


In [11]:
for book, link in book_characters.items():
    print(book)
    dir_name = book.split('.')[0]
    os.makedirs(dir_name, exist_ok=True)

    pairwise_characters = get_pairwise_characters(link)

    with open(book, 'r') as file:
        content = file.read()
        content = content.replace("\n", " ")
        content = content.replace('\u00A0', ' ').strip()
        content = content.replace('"', ' ')
        content = content.replace("Witcher", "Geralt").replace("witcher", "Geralt")
        content = content.replace("Cirilla", "Ciri")
        sentences = re.split(r'(?<=[.!?])\s+', content)

    for pair in pairwise_characters:
        story = []
        weight = 0
        idx = 0

        while idx < len(sentences):
            if pair[0] in sentences[idx]:
                if pair[1] in " ".join(sentences[idx-2:idx+3]):
                    weight += 1
                    story.append(" ".join(sentences[idx-2:idx+3]))
                    idx +=4
                else:
                    idx +=1
            else:
                idx +=1



        if weight > 10:
            file_name = pair[0] + '_' + pair[1] + '_' + '.csv'
            file_path = os.path.join(dir_name, file_name)
            story_to_save = pd.DataFrame(story, columns = ['phrases'])
            story_to_save.to_csv(file_path, index = False)



Blood of Elves.txt
Times of Contempt.txt
Baptism of Fire.txt
The Tower of the Swallow.txt
The Lady of the Lake.txt


In [9]:
sentiment_pipeline = pipeline("sentiment-analysis")
emotion_pipeline = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
def get_labels(df):

    sentiment = sentiment_pipeline(list(df['phrases']))
    phrase_sentiment = pd.concat([df, pd.DataFrame(sentiment)['label']], axis=1)
    #print(phrase_sentiment.head())

    phrase_emotion = emotion_pipeline(list(df['phrases']) )
    df_emotion = pd.DataFrame([max(i, key=lambda x: x['score'])["label"] for i in phrase_emotion], columns = ['emotion'])

    df_full = pd.concat([phrase_sentiment, df_emotion] , axis=1)

    return df_full

In [12]:
folders = [
    "Blood of Elves",
    "Times of Contempt",
    "Baptism of Fire",
    "The Tower of the Swallow",
    "The Lady of the Lake"]

df_final = pd.DataFrame()

classify = True
final_output = "witcher_network_classified.csv"

for folder_path in folders:
    for filename in os.listdir(folder_path):
        print(filename)
        #if os.path.isfile(os.path.join(folder_path, filename)):

        pair = pd.read_csv(os.path.join(folder_path, filename))

        if classify:
            story = get_labels(pair)
            #story.to_csv(os.path.join(folder_path, filename), index = False)
        else:
            story = pair.copy()

        meta_df = pd.DataFrame({"book" : [folder_path],
                        "person_1" : [filename.split('_')[0]],
                        "person_2" : [filename.split('_')[1]],
                        })

        story['key'] = 0
        meta_df['key'] = 0
        pair_df_meta = pd.merge(story, meta_df, on='key').drop('key', axis=1)

        df_final = pd.concat([df_final, pair_df_meta], axis=0, ignore_index=True)
    #     break
    # break


df_final.to_csv(final_output, index = False)
df_final.shape

Ciri_Calanthe_.csv
                                             phrases     label
0  'Queen Calanthe, the Lionness of Cintra, had a...  NEGATIVE
1  But the Geralt  could neither find her nor col...  NEGATIVE
2  If I am not mistaken, the true  destinies of r...  POSITIVE
3  So I advise you to weigh each word with  care,...  POSITIVE
4  On these windmills and pendulums of yours, on ...  POSITIVE
Vilgefortz_Lydia_.csv
                                             phrases     label
0  'Two proved enough. But still complicated, I m...  POSITIVE
1  Tissaia de Vries ate slowly, devoting more att...  NEGATIVE
2  The  Chapter wants to assume a unanimous stand...  POSITIVE
3  It served for the detection of persons who  ha...  NEGATIVE
4  Lydia glanced at Vilgefortz, caught the sign t...  NEGATIVE
Geralt_Calanthe_.csv
                                             phrases     label
0  'Calanthe's granddaughter. She was not the pri...  NEGATIVE
1  Before throwing herself from the  tower, Queen...  PO

(4871, 6)